# Experiment 1: Embedded variable selection algorithms testing

### (1) Importing and defining all required functions

In [1]:
"""
Importing required libraries and defining key functions
"""
import numpy as np
from functools import partial
import time
from IPython.display import clear_output
import inspect
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Algorithm functions
import os
os.chdir('C:/Users/hughw/Documents/MSC project/GP algorithms/Master function files')
from GP_funcs_ZTMFSS import kernel_funcs
from GP_funcs_ZTMFSS import draw_GP
from GP_funcs_ZTMFSS import simulations
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
os.chdir('C:/Users/hughw/Documents/MSC project/Simulation results')

### (2) Setting simulation parameters and models

In [2]:
"""
Importing in rpy2
"""

os.environ['R_USER'] = 'D:\Anaconda3\Lib\site-packages\rpy2'
import rpy2
print(rpy2.__version__)
import rpy2.robjects as robjects

from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')
base.R_home()
# import R's "utils" package
utils = importr('utils')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

3.4.5


<rpy2.rinterface_lib.sexp.NULLType object at 0x00000251D2DE47C0> [RTYPES.NILSXP]

In [3]:
"""
Simulation controls
"""
# Simulation settings
n=300
ntest=100
p=100
q=5
corr=0.5
r2=0.9
sigma_X=1
strue=1
lsmean=0.25 # toggle between 0.25 and 1
ltrue =np.repeat(lsmean/q**0.5,p)
kern = kernel_funcs.gaussian
nruns = 100
m = 9


# Storage objects
Runtime=np.zeros((nruns, m))
MSE_F=np.zeros((nruns, m))
MSE_Y=np.zeros((nruns,m))
TPR=np.zeros((nruns,m))
TNR=np.zeros((nruns,m))
PPV=np.zeros((nruns,m))
NPV=np.zeros((nruns,m))
MCC=np.zeros((nruns,m))
RF_oob = np.zeros((nruns,3))

### (3) Running algorithms

In [5]:
np.random.seed(8750)
runlist = np.random.choice(1000,100,False) # Choose 100 random trials
othermodels=True
SBGAM=False

for run in range(len(runlist)):
    
    """
    Generating data and scaling data
    """
    lselect=[]
    np.random.seed(runlist[run]) # Fixing trial seed
    t=time.time()
    Y,F,X,e,lselect,strue,sigma,select=draw_GP.draw_GP_ARD_lm(n,ntest,p,q,sigma_X,corr,strue,ltrue,plot_YX=True,kern=kern,cop=False,r2=r2)
    
    Y = Y.reshape(n+ntest,1)
    F = F.reshape(n+ntest,1)
    
    Y = (Y-Y.mean())/Y.var()**0.5
    F = (F-F.mean())/F.var()**0.5
    X = (X-X.mean(0))/X.var(0)**0.5

    # Getting training and test set
    ytest=Y[n:]
    Xtest=X[n:]
    ftest=F[n:]
    y=Y[:n]
    X=X[:n]
    f=F[:n]
    print("data generated")
    print("Noise variance is: ",sigma**2)
    print("Average data variance is: ", np.mean(np.var(X,0)))
    print(time.time()-t)
    
    if othermodels:
    
        """
        Running Random forest with VS+CV
        """
        
        # Setting up storage objects
        folds=5
        thresholds = ["1*mean", "2*mean", "4*mean", "8*mean", "16*mean"]
        errcv = np.zeros(len(thresholds))
        Selected_features = []
        
        t = time.time()
        
        # Shuffling the data and splitting into folds
        shuffled_indexes = np.random.choice(n,n,False)
        y_shuffle = y[shuffled_indexes]
        X_shuffle = X[shuffled_indexes]
        n_per_fold = int(n/folds)
        
        # Doing CV over thresholds
        for i in range(len(thresholds)):
            
            # Getting selected features          
            embedded_rf_selector = SelectFromModel(RandomForestRegressor(n_estimators=100), threshold = thresholds[i])
            embedded_rf_selector.fit(X, y.reshape(n,))
            embedded_rf_support = embedded_rf_selector.get_support()
            print(str(np.sum(embedded_rf_support)), 'selected features')
            selected_features = np.where(embedded_rf_support)[0]
            if len(selected_features)==0:
                selected_features = np.random.choice(p,1,False)
            Selected_features.append(selected_features)

            for f in range(folds):

                # Getting CVtraining and CVtest set
                y_cvtest = y_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]
                X_cvtest = X_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]

                if f>0:
                    y_cvtrain = y_shuffle[:(f*n_per_fold)]
                    X_cvtrain = X_shuffle[:(f*n_per_fold)]
                if f<(folds-1):
                    if f>0:
                        y_cvtrain = np.append(y_cvtrain,y_shuffle[(f+1)*n_per_fold:],0)
                        X_cvtrain = np.append(X_cvtrain,X_shuffle[(f+1)*n_per_fold:],0)
                    else:
                        y_cvtrain = y_shuffle[(f+1)*n_per_fold:]
                        X_cvtrain = X_shuffle[(f+1)*n_per_fold:]

                # Running training and predicting with RF
                model = RandomForestRegressor(n_estimators=100).fit(X_cvtrain[:,selected_features],y_cvtrain.reshape(len(y_cvtrain),))
                preds = model.predict(X_cvtest[:,selected_features])
                errcv[i] += np.mean((y_cvtest - preds)**2)
        
        # Getting best threshold and running final model
        best_threshold =np.where(errcv==np.min(errcv))[0][0]
        model = RandomForestRegressor(n_estimators=100).fit(X[:,Selected_features[best_threshold]],y.reshape(n,))
        preds = model.predict(Xtest[:,Selected_features[best_threshold]])
        Runtime[run,0] = time.time()-t
        
        # MSE
        MSE_Y[run,0] = simulations.MSE_pc(preds.reshape(ntest,1),ytest)
        MSE_F[run,0] = simulations.MSE_pc(preds.reshape(ntest,1),ftest)

        # VS accuracy
        forest_select = np.zeros(p)
        forest_select[Selected_features[best_threshold]]=1
        PPV[run,0]=np.mean(select[forest_select>0])
        NPV[run,0]=np.mean((1-select[forest_select==0]))
        TPR[run,0]=np.mean(forest_select[select>0])
        TNR[run,0]=np.mean((1-forest_select[select==0]))

        TP = np.sum(forest_select[select>0])
        TN = np.sum(1-forest_select[select==0])
        FP = np.sum(1-select[forest_select>0])
        FN = np.sum(select[forest_select==0])

        MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

        if (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)==0:
            MCC[run,0]=0


        """
        Running Gboost with VS-CV
        """

        # Setting up storage objects
        folds=5
        thresholds = ["0.00625*mean", "0.125*mean", "0.25*mean", "0.5*mean", "1*mean", "2*mean", "4*mean", "8*mean", "16*mean"]
        errcv = np.zeros(len(thresholds))
        Selected_features = []
        
        t = time.time()
        
        # Shuffling the data and splitting into folds
        shuffled_indexes = np.random.choice(n,n,False)
        y_shuffle = y[shuffled_indexes]
        X_shuffle = X[shuffled_indexes]
        n_per_fold = int(n/folds)
        
        # Doing CV over thresholds
        for i in range(len(thresholds)):
            
            # Getting selected features          
            embedded_rf_selector = SelectFromModel(GradientBoostingRegressor(n_estimators=100,learning_rate=0.01), threshold = thresholds[i])
            embedded_rf_selector.fit(X, y.reshape(n,))
            embedded_rf_support = embedded_rf_selector.get_support()
            print(str(np.sum(embedded_rf_support)), 'selected features')
            selected_features = np.where(embedded_rf_support)[0]
            if len(selected_features)==0:
                selected_features = np.random.choice(p,1,False)
            Selected_features.append(selected_features)

            for f in range(folds):

                # Getting CVtraining and CVtest set
                y_cvtest = y_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]
                X_cvtest = X_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]

                if f>0:
                    y_cvtrain = y_shuffle[:(f*n_per_fold)]
                    X_cvtrain = X_shuffle[:(f*n_per_fold)]
                if f<(folds-1):
                    if f>0:
                        y_cvtrain = np.append(y_cvtrain,y_shuffle[(f+1)*n_per_fold:],0)
                        X_cvtrain = np.append(X_cvtrain,X_shuffle[(f+1)*n_per_fold:],0)
                    else:
                        y_cvtrain = y_shuffle[(f+1)*n_per_fold:]
                        X_cvtrain = X_shuffle[(f+1)*n_per_fold:]

                # Running training and predicting with RF
                model = GradientBoostingRegressor(n_estimators=100,learning_rate=0.01).fit(X_cvtrain[:,selected_features],y_cvtrain.reshape(len(y_cvtrain),))
                preds = model.predict(X_cvtest[:,selected_features])
                errcv[i] += np.mean((y_cvtest - preds)**2)
        
        # Getting best threshold and running final model
        best_threshold =np.where(errcv==np.min(errcv))[0][0]
        model = GradientBoostingRegressor(n_estimators=100,learning_rate=0.01).fit(X[:,Selected_features[best_threshold]],y.reshape(n,))
        preds = model.predict(Xtest[:,Selected_features[best_threshold]])
        Runtime[run,1] = time.time()-t
        
        # MSE
        MSE_Y[run,1] = simulations.MSE_pc(preds.reshape(ntest,1),ytest)
        MSE_F[run,1] = simulations.MSE_pc(preds.reshape(ntest,1),ftest)

        # VS accuracy
        forest_select = np.zeros(p)
        forest_select[Selected_features[best_threshold]]=1
        PPV[run,1]=np.mean(select[forest_select>0])
        NPV[run,1]=np.mean((1-select[forest_select==0]))
        TPR[run,1]=np.mean(forest_select[select>0])
        TNR[run,1]=np.mean((1-forest_select[select==0]))

        TP = np.sum(forest_select[select>0])
        TN = np.sum(1-forest_select[select==0])
        FP = np.sum(1-select[forest_select>0])
        FN = np.sum(select[forest_select==0])

        MCC[run,1]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

        if (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)==0:
            MCC[run,1]=0


    """
    Printing out current results
    """
    print("RUN {0}".format(run))
    print("Runtime mean is:", Runtime[:run+1].mean(0))
    print("TPR mean is:", TPR[:run+1].mean(0))
    print("PPV mean is:", PPV[:run+1].mean(0))
    print("MCC mean is:", MCC[:run+1].mean(0))
    print("MSE_F mean is:", MSE_F[:run+1].mean(0))
    print("MSE_Y mean is:", MSE_Y[:run+1].mean(0), "\n")

R2= 0.8993636862772934
data generated
Noise variance is:  0.006067570867112703
Average data variance is:  0.9972629072424785
0.03487586975097656
100 selected features
97 selected features
42 selected features
11 selected features
4 selected features
3 selected features
2 selected features
2 selected features
2 selected features
21 selected features
11 selected features
9 selected features
6 selected features
5 selected features
4 selected features
3 selected features
2 selected features
2 selected features
RUN 0
Runtime mean is: [16.53687048  5.64980245  0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [1.  0.6 0.  0.  0.  0.  0.  0.  0. ]
PPV mean is: [0.05154639 1.         0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.04034577 0.76656954 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.27604646 0.4031223  0.         0.         0.         0.
 0.         0.         

C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-5-a482031c6ffb>:113: RuntimeWarning: invalid value encountered in double_scalars
  MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


27 selected features
13 selected features
7 selected features
3 selected features
2 selected features
2 selected features
2 selected features
2 selected features
2 selected features
RUN 1
Runtime mean is: [16.09138775  5.73203564  0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [1.  0.6 0.  0.  0.  0.  0.  0.  0. ]
PPV mean is: [0.0507732  0.71428571 0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.02017288 0.62155983 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.21078181 0.35349458 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_Y mean is: [0.31391444 0.41780773 0.         0.         0.         0.
 0.         0.         0.        ] 

R2= 0.9080835121498346
data generated
Noise variance is:  0.0054187767092858845
Average data variance is:  1.012614884699872
0.03271150588989258
100 selected features
99 selected features
57 selected feature

C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-5-a482031c6ffb>:113: RuntimeWarning: invalid value encountered in double_scalars
  MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


23 selected features
17 selected features
11 selected features
7 selected features
3 selected features
3 selected features
2 selected features
2 selected features
2 selected features
RUN 4
Runtime mean is: [16.33247108  5.76737924  0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [0.96 0.6  0.   0.   0.   0.   0.   0.   0.  ]
PPV mean is: [0.05331351 0.64047619 0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.02969582 0.55879555 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.21034144 0.37708304 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_Y mean is: [0.31114326 0.43524984 0.         0.         0.         0.
 0.         0.         0.        ] 

R2= 0.9014392609465416
data generated
Noise variance is:  0.008110831557296446
Average data variance is:  1.0078797080723603
0.033643484115600586
100 selected features
96 selected features
35 selec

...

21 selected features
12 selected features
8 selected features
7 selected features
4 selected features
4 selected features
4 selected features
4 selected features
2 selected features
RUN 96
Runtime mean is: [17.12142628  6.0712757   0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [0.98762887 0.79381443 0.         0.         0.         0.
 0.         0.         0.        ]
PPV mean is: [0.05330454 0.40670459 0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.02919062 0.47091839 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.22007489 0.3924549  0.         0.         0.         0.
 0.         0.         0.        ]
MSE_Y mean is: [0.29585951 0.4386573  0.         0.         0.         0.
 0.         0.         0.        ] 

R2= 0.8980860028775616
data generated
Noise variance is:  0.004503056471917673
Average data variance is:  0.9984008731786534
0.031722068786621094
100 

C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-5-a482031c6ffb>:113: RuntimeWarning: invalid value encountered in double_scalars
  MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


52 selected features
15 selected features
12 selected features
10 selected features
6 selected features
5 selected features
4 selected features
3 selected features
3 selected features
RUN 97
Runtime mean is: [17.14262597  6.08402557  0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [0.9877551  0.79591837 0.         0.         0.         0.
 0.         0.         0.        ]
PPV mean is: [0.05327082 0.4035357  0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.02889275 0.46836224 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.22181713 0.39419951 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_Y mean is: [0.2975877  0.44028974 0.         0.         0.         0.
 0.         0.         0.        ] 

R2= 0.9060210673725113
data generated
Noise variance is:  0.022898177536325882
Average data variance is:  0.9815890054080494
0.0362391471862793
100 

C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-5-a482031c6ffb>:113: RuntimeWarning: invalid value encountered in double_scalars
  MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


34 selected features
16 selected features
10 selected features
5 selected features
3 selected features
3 selected features
3 selected features
2 selected features
1 selected features
RUN 98
Runtime mean is: [17.13658993  6.08622668  0.          0.          0.          0.
  0.          0.          0.        ]
TPR mean is: [0.98787879 0.7959596  0.         0.         0.         0.
 0.         0.         0.        ]
PPV mean is: [0.05323779 0.40064794 0.         0.         0.         0.
 0.         0.         0.        ]
MCC mean is: [0.02860091 0.46588158 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_F mean is: [0.22095597 0.39333322 0.         0.         0.         0.
 0.         0.         0.        ]
MSE_Y mean is: [0.29670551 0.43937832 0.         0.         0.         0.
 0.         0.         0.        ] 

R2= 0.900112364354712
data generated
Noise variance is:  0.004758340715958492
Average data variance is:  1.010378106428452
0.03362727165222168
100 se

In [10]:
np.random.seed(8750)
runlist = np.random.choice(1000,100,False) # Choose 100 random trials
othermodels=False
SBGAM=True

for run in range(len(runlist)):
    
    """
    Generating data and scaling data
    """
    lselect=[]
    np.random.seed(runlist[run]) # Fixing trial seed
    t=time.time()
    Y,F,X,e,lselect,strue,sigma,select=draw_GP.draw_GP_ARD_lm(n,ntest,p,q,sigma_X,corr,strue,ltrue,plot_YX=True,kern=kern,cop=False,r2=r2)
    
    Y = Y.reshape(n+ntest,1)
    F = F.reshape(n+ntest,1)
    
    Y = (Y-Y.mean())/Y.var()**0.5
    F = (F-F.mean())/F.var()**0.5
    X = (X-X.mean(0))/X.var(0)**0.5

    # Getting training and test set
    ytest=Y[n:]
    Xtest=X[n:]
    ftest=F[n:]
    y=Y[:n]
    X=X[:n]
    f=F[:n]
    print("data generated")
    print("Noise variance is: ",sigma**2)
    print("Average data variance is: ", np.mean(np.var(X,0)))
    print(time.time()-t)
    
    """
    Exporting data to enable running of R scripts
    """
    os.chdir('C:/Users/hughw/Documents/MSC project/R scripts')
    np.save("y", y)
    np.save("f", f)
    np.save("X", X)
    np.save("ytest", ytest)
    np.save("ftest", ftest)
    np.save("Xtest", Xtest)
    np.save("select", select.astype(float))   
    
    if othermodels:
    
        """
        Running Random forest with VS+CV
        """
        
        # Setting up storage objects
        folds=5
        thresholds = ["1*mean", "2*mean", "4*mean", "8*mean", "16*mean"]
        errcv = np.zeros(len(thresholds))
        Selected_features = []
        
        t = time.time()
        
        # Shuffling the data and splitting into folds
        shuffled_indexes = np.random.choice(n,n,False)
        y_shuffle = y[shuffled_indexes]
        X_shuffle = X[shuffled_indexes]
        n_per_fold = int(n/folds)
        
        # Doing CV over thresholds
        for i in range(len(thresholds)):
            
            # Getting selected features          
            embedded_rf_selector = SelectFromModel(RandomForestRegressor(n_estimators=100), threshold = thresholds[i])
            embedded_rf_selector.fit(X, y.reshape(n,))
            embedded_rf_support = embedded_rf_selector.get_support()
            print(str(np.sum(embedded_rf_support)), 'selected features')
            selected_features = np.where(embedded_rf_support)[0]
            if len(selected_features)==0:
                selected_features = np.random.choice(p,1,False)
            Selected_features.append(selected_features)

            for f in range(folds):

                # Getting CVtraining and CVtest set
                y_cvtest = y_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]
                X_cvtest = X_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]

                if f>0:
                    y_cvtrain = y_shuffle[:(f*n_per_fold)]
                    X_cvtrain = X_shuffle[:(f*n_per_fold)]
                if f<(folds-1):
                    if f>0:
                        y_cvtrain = np.append(y_cvtrain,y_shuffle[(f+1)*n_per_fold:],0)
                        X_cvtrain = np.append(X_cvtrain,X_shuffle[(f+1)*n_per_fold:],0)
                    else:
                        y_cvtrain = y_shuffle[(f+1)*n_per_fold:]
                        X_cvtrain = X_shuffle[(f+1)*n_per_fold:]

                # Running training and predicting with RF
                model = RandomForestRegressor(n_estimators=100).fit(X_cvtrain[:,selected_features],y_cvtrain.reshape(len(y_cvtrain),))
                preds = model.predict(X_cvtest[:,selected_features])
                errcv[i] += np.mean((y_cvtest - preds)**2)
        
        # Getting best threshold and running final model
        best_threshold =np.where(errcv==np.min(errcv))[0][0]
        model = RandomForestRegressor(n_estimators=100).fit(X[:,Selected_features[best_threshold]],y.reshape(n,))
        preds = model.predict(Xtest[:,Selected_features[best_threshold]])
        Runtime[run,0] = time.time()-t
        
        # MSE
        MSE_Y[run,0] = simulations.MSE_pc(preds.reshape(ntest,1),ytest)
        MSE_F[run,0] = simulations.MSE_pc(preds.reshape(ntest,1),ftest)

        # VS accuracy
        forest_select = np.zeros(p)
        forest_select[Selected_features[best_threshold]]=1
        PPV[run,0]=np.mean(select[forest_select>0])
        NPV[run,0]=np.mean((1-select[forest_select==0]))
        TPR[run,0]=np.mean(forest_select[select>0])
        TNR[run,0]=np.mean((1-forest_select[select==0]))

        TP = np.sum(forest_select[select>0])
        TN = np.sum(1-forest_select[select==0])
        FP = np.sum(1-select[forest_select>0])
        FN = np.sum(select[forest_select==0])

        MCC[run,0]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

        if (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)==0:
            MCC[run,0]=0


        """
        Running Gboost with VS-CV
        """

        # Setting up storage objects
        folds=5
        thresholds = ["1*mean", "2*mean", "4*mean", "8*mean", "16*mean"]
        errcv = np.zeros(len(thresholds))
        Selected_features = []
        
        t = time.time()
        
        # Shuffling the data and splitting into folds
        shuffled_indexes = np.random.choice(n,n,False)
        y_shuffle = y[shuffled_indexes]
        X_shuffle = X[shuffled_indexes]
        n_per_fold = int(n/folds)
        
        # Doing CV over thresholds
        for i in range(len(thresholds)):
            
            # Getting selected features          
            embedded_rf_selector = SelectFromModel(GradientBoostingRegressor(n_estimators=100,learning_rate=0.01), threshold = thresholds[i])
            embedded_rf_selector.fit(X, y.reshape(n,))
            embedded_rf_support = embedded_rf_selector.get_support()
            print(str(np.sum(embedded_rf_support)), 'selected features')
            selected_features = np.where(embedded_rf_support)[0]
            if len(selected_features)==0:
                selected_features = np.random.choice(p,1,False)
            Selected_features.append(selected_features)

            for f in range(folds):

                # Getting CVtraining and CVtest set
                y_cvtest = y_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]
                X_cvtest = X_shuffle[(f*n_per_fold):min((f+1)*n_per_fold, n)]

                if f>0:
                    y_cvtrain = y_shuffle[:(f*n_per_fold)]
                    X_cvtrain = X_shuffle[:(f*n_per_fold)]
                if f<(folds-1):
                    if f>0:
                        y_cvtrain = np.append(y_cvtrain,y_shuffle[(f+1)*n_per_fold:],0)
                        X_cvtrain = np.append(X_cvtrain,X_shuffle[(f+1)*n_per_fold:],0)
                    else:
                        y_cvtrain = y_shuffle[(f+1)*n_per_fold:]
                        X_cvtrain = X_shuffle[(f+1)*n_per_fold:]

                # Running training and predicting with RF
                model = GradientBoostingRegressor(n_estimators=100,learning_rate=0.01).fit(X_cvtrain[:,selected_features],y_cvtrain.reshape(len(y_cvtrain),))
                preds = model.predict(X_cvtest[:,selected_features])
                errcv[i] += np.mean((y_cvtest - preds)**2)
        
        # Getting best threshold and running final model
        best_threshold =np.where(errcv==np.min(errcv))[0][0]
        model = GradientBoostingRegressor(n_estimators=100,learning_rate=0.01).fit(X[:,Selected_features[best_threshold]],y.reshape(n,))
        preds = model.predict(Xtest[:,Selected_features[best_threshold]])
        Runtime[run,1] = time.time()-t
        
        # MSE
        MSE_Y[run,1] = simulations.MSE_pc(preds.reshape(ntest,1),ytest)
        MSE_F[run,1] = simulations.MSE_pc(preds.reshape(ntest,1),ftest)

        # VS accuracy
        forest_select = np.zeros(p)
        forest_select[Selected_features[best_threshold]]=1
        PPV[run,1]=np.mean(select[forest_select>0])
        NPV[run,1]=np.mean((1-select[forest_select==0]))
        TPR[run,1]=np.mean(forest_select[select>0])
        TNR[run,1]=np.mean((1-forest_select[select==0]))

        TP = np.sum(forest_select[select>0])
        TN = np.sum(1-forest_select[select==0])
        FP = np.sum(1-select[forest_select>0])
        FN = np.sum(select[forest_select==0])

        MCC[run,1]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

        if (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)==0:
            MCC[run,1]=0
            
        """
        Running ncvreg
        """
        r=robjects.r
        output = r.source("ncvreg_code.R")
        Runtime[run,2:5]=output[0][0]
        MSE_F[run,2:5]=output[0][1]
        MSE_Y[run,2:5]=output[0][2]
        TPR[run,2:5]=output[0][3]
        TNR[run,2:5]=output[0][4]
        PPV[run,2:5]=output[0][5]
        NPV[run,2:5]=output[0][6]
        MCC[run,2:5]=output[0][7]


        """
        Running sfgam
        """
        r=robjects.r
        output = r.source("sparseGAM_code.R")
        Runtime[run,5:8]=output[0][0]
        MSE_F[run,5:8]=output[0][1]
        MSE_Y[run,5:8]=output[0][2]
        TPR[run,5:8]=output[0][3]
        TNR[run,5:8]=output[0][4]
        PPV[run,5:8]=output[0][5]
        NPV[run,5:8]=output[0][6]
        MCC[run,5:8]=output[0][7]

    """
    Runnig sbgam
    """
    if SBGAM:
        r=robjects.r
        output = r.source("sparseBayesGAM_code.R")
        Runtime[run,8]=np.array(output[0][0])
        MSE_F[run,8]=np.array(output[0][1])
        MSE_Y[run,8]=np.array(output[0][2])
        TPR[run,8]=np.array(output[0][3])
        TNR[run,8]=np.array(output[0][4])
        PPV[run,8]=np.array(output[0][5])
        NPV[run,8]=np.array(output[0][6])
        MCC[run,8]=np.array(output[0][7])
    
    """
    Removing files from directory
    """
    os.remove("y.npy")
    os.remove("f.npy")
    os.remove("X.npy")
    os.remove("ytest.npy")
    os.remove("ftest.npy")
    os.remove("Xtest.npy")
    os.remove("select.npy")

    """
    Printing out current results
    """
    print("RUN {0}".format(run))
    print("Runtime mean is:", Runtime[:run+1].mean(0))
    print("TPR mean is:", TPR[:run+1].mean(0))
    print("PPV mean is:", PPV[:run+1].mean(0))
    print("MCC mean is:", MCC[:run+1].mean(0))
    print("MSE_F mean is:", MSE_F[:run+1].mean(0))
    print("MSE_Y mean is:", MSE_Y[:run+1].mean(0), "\n")
    

R2= 0.8993636862772934
data generated
Noise variance is:  0.006067570867112703
Average data variance is:  0.9972629072424785
0.07080984115600586
Fold number 1 
Fold number 2 
Fold number 3 
lambda0 =  60 
RUN 0
Runtime mean is: [0.         0.         0.         0.         0.         0.
 0.         0.         1.05049987]
TPR mean is: [0.  0.  0.  0.  0.  0.  0.  0.  0.6]
PPV mean is: [0. 0. 0. 0. 0. 0. 0. 0. 1.]
MCC mean is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.76656954]
MSE_F mean is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.23702995]
MSE_Y mean is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.36967159] 

R2= 0.9021920644185265
data generated
Noise variance is:  0.0056107843331200755
Average data variance is:  1.0170416340411146
0.059836626052856445
Fold number 1 
Fold number 2 
Fold number 3 
lambda0 =  60 
RUN 1
Runtime mean is: [0.         0.         0.   

In [12]:
namelist = ["Runtime", "MSE_F", "MSE_Y", "TPR", "TNR", "PPV", "NPV", "MCC"]
objlist = [Runtime, MSE_F, MSE_Y, TPR, TNR, PPV, NPV, MCC]
#iters = np.random.choice(1000,100,False)
iters = np.linspace(0,99,100).astype(int)

for i in range(len(objlist)):
    print("Mean {0} is:".format(namelist[i]), np.mean(objlist[i][iters],0))

print("\n")
for i in range(len(objlist)):
    print("Median {0} is:".format(namelist[i]), np.median(objlist[i][iters],0))

print("\n")
quant = 0.25
for i in range(len(objlist)):
    if namelist[i] in ["Runtime", "MSE_F", "MSE_Y"]:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],1-quant,0))
    else:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],quant,0))

Mean Runtime is: [ 0.          0.          0.          0.          0.          0.
  0.          0.         36.37381007]
Mean MSE_F is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.24034256]
Mean MSE_Y is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.34837011]
Mean TPR is: [0.    0.    0.    0.    0.    0.    0.    0.    0.606]
Mean TNR is: [0. 0. 0. 0. 0. 0. 0. 0. 1.]
Mean PPV is: [0. 0. 0. 0. 0. 0. 0. 0. 1.]
Mean NPV is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.97978341]
Mean MCC is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.76041299]


Median Runtime is: [ 0.          0.          0.          0.          0.          0.
  0.          0.         39.97057593]
Median MSE_F is: [0.         0.         0.         0.         0.         0.
 0.         0.         0.19172483]
Median MSE_Y is: [0.         0.         0.         0.       

# REMEMBER TO MULTIPLY RUNTIME BY 60

In [13]:
Output = {"Runtime" : Runtime, "MSE_F" : MSE_F
        , "MSE_Y" : MSE_Y, "TPR" :TPR, "TNR" : TNR, "PPV" : PPV, "NPV" : NPV, "MCC" : MCC}
String = "Stage1_GP_lsmean={0}_p={1}_embedded_sslasso".format(lsmean,p)
np.save(String, Output) # saving